In [1]:
# import packages
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pdz
import numpy as np
import pandas as pd

from pandas import Series, DataFrame
import Bio
from Bio import SeqIO,AlignIO

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.mixture import GaussianMixture as GMM

from sklearn.manifold import TSNE
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


In [22]:
# methods

def parseFasta(data):
    d = {fasta.id : str(fasta.seq) for fasta in SeqIO.parse(data, "fasta")}
    pd.DataFrame([d])
    s = pd.Series(d, name='Sequence')
    s.index.name = 'ID'
    s.reset_index()
    return pd.DataFrame(s)

def get_kmer_table(path,k_min,k_max):
    genes,gene_len = read_fasta(path)
    count_vect = CountVectorizer(analyzer='char', ngram_range=(k_min, k_max))
    X = count_vect.fit_transform(genes)
    chars = count_vect.get_feature_names()
    kmers = X.toarray()
    kmer_freq = []
    for i in range(len(genes)):
        kmer_freq.append(kmers[i] / gene_len[i])
    input = pd.DataFrame(kmer_freq, columns=chars)
    return input

def get_gene_sequences(filename):
    genes = []
    for record in SeqIO.parse(filename, "fasta"):
        genes.append(str(record.seq))
    return genes

# genes: a list of gene sequences, which can directly be generated from get_gene_sequences().
def get_gene_len(genes):
    gene_len = []

    for i in range(len(genes)):
        gene_len.append(len(genes[i]))
    return gene_len

#read single fasta file containing all the gene sequences
def read_fasta(path):
    virus = parseFasta(path)
    virus = virus.drop_duplicates(keep="last")
    genes = list(virus['Sequence'])
    gene_seq = get_gene_sequences(path)
    gene_len = get_gene_len(gene_seq)
    return gene_seq,gene_len

def get_predictions_default(path,k_min,k_max,num_class,cov_type):
    seed  = np.random.seed(None)
    ran_state = np.random.get_state()
    kmer_table = get_kmer_table(path, k_min, k_max)
    gmm = GMM(n_components=num_class,covariance_type=cov_type,random_state = seed).fit(kmer_table)
    labels = gmm.predict(kmer_table)
    return labels,ran_state

def get_predictions_from_state(path,k_min,k_max,num_class,cov_type,state):
    kmer_table = get_kmer_table(path, k_min, k_max)
    gmm = GMM(n_components=num_class,covariance_type=cov_type,random_state = np.random.set_state(state)).fit(kmer_table)
    labels = gmm.predict(kmer_table)
    return labels

def get_predictions(path,k_min,k_max,num_class,cov_type, seed):
    kmer_table = get_kmer_table(path, k_min, k_max)
    gmm = GMM(n_components=num_class,covariance_type=cov_type,random_state = seed).fit(kmer_table)
    labels = gmm.predict(kmer_table)
    return labels

def cal_accuracy(labels, predictions):
    err = 0
    total_len = len(labels)
    for i in range(len(labels)):
        if (labels[i] == -1):
            total_len = total_len-1
            continue
        if (labels[i] != predictions[i]):
            err += 1
            
    return 1-err/(total_len)

def get_predictions_semi(path,k_min,k_max,num_class,cov_type,seed,labels):
    kmer_table = get_kmer_table(path, k_min, k_max)
    finalDf = pd.concat([kmer_table, pd.Series(labels)], axis = 1)
    gmm = GMM(n_components=num_class,covariance_type=cov_type,random_state = seed)
    gmm.means_init = np.array([kmer_table[finalDf.Labels == i].mean(axis=0) for i in range(num_class)])
    gmm.fit(kmer_table)
    predictions = gmm.predict(kmer_table)
    return predictions


In [51]:
def model_selection(path,labels,num_class):
    best_accu = 0
    best_prediction = []
    cov_type = ['full','diag','tied','spherical']
    k_min = [2,3,4]
    k_max = [3,4,5]
    for cov in cov_type:
        for k1 in k_min:
            for k2 in k_max:
                if (k2 >= k1):
                    prediction = get_predictions_semi(path,k1,k2,num_class,cov,0,labels)
                    accu = cal_accuracy(labels,prediction)
                    if accu > best_accu: 
                        best_accu = accu
                        best_kmin = k1
                        best_kmax = k2
                        best_cov = cov
                        best_prediction = prediction
    print('Best model has the following parameters:')
    print('minimum length of kmer: ', best_kmin)
    print('maximum length of kmer: ', best_kmax)
    print('covariance type: ', best_cov)
    print('It has an accuracy regard to known labels of ',best_accu)
    return best_prediction

In [12]:
def sammon(x, n, display = 2, maxhalves = 20, maxiter = 500, tolfun = 1e-9):

    import numpy as np 
    from scipy.spatial.distance import cdist

    D = cdist(x, x) # distance matrix
    init = 'pca' # initialization from pca

    if np.count_nonzero(np.diagonal(D)) > 0:
        raise ValueError("The diagonal of the dissimilarity matrix must be zero")

    # Remaining initialisation
    N = x.shape[0]
    scale = 0.5 / D.sum()
    D = D + np.eye(N)     

    if np.count_nonzero(D<=0) > 0:
        raise ValueError("Off-diagonal dissimilarities must be strictly positive")   

    Dinv = 1 / D
    [UU,DD,_] = np.linalg.svd(x)
    y = UU[:,:n]*DD[:n] 
    
    one = np.ones([N,n])
    d = cdist(y,y) + np.eye(N)
    dinv = 1. / d
    delta = D-d 
    E = ((delta**2)*Dinv).sum() 

    for i in range(maxiter):

        # Compute gradient, Hessian and search direction (note it is actually
        # 1/4 of the gradient and Hessian, but the step size is just the ratio
        # of the gradient and the diagonal of the Hessian so it doesn't
        # matter).
        delta = dinv - Dinv
        deltaone = np.dot(delta,one)
        g = np.dot(delta,y) - (y * deltaone)
        dinv3 = dinv ** 3
        y2 = y ** 2
        H = np.dot(dinv3,y2) - deltaone - np.dot(2,y) * np.dot(dinv3,y) + y2 * np.dot(dinv3,one)
        s = -g.flatten(order='F') / np.abs(H.flatten(order='F'))
        y_old    = y

        # Use step-halving procedure to ensure progress is made
        for j in range(maxhalves):
            s_reshape = np.reshape(s, (-1,n),order='F')
            y = y_old + s_reshape
            d = cdist(y, y) + np.eye(N)
            dinv = 1 / d
            delta = D - d
            E_new = ((delta**2)*Dinv).sum()
            if E_new < E:
                break
            else:
                s = 0.5*s

        # Bomb out if too many halving steps are required
        if j == maxhalves-1:
            print('Warning: maxhalves exceeded. Sammon mapping may not converge...')

        # Evaluate termination criterion
        if abs((E - E_new) / E) < tolfun:
            if display:
                print('TolFun exceeded: Optimisation terminated')
            break

        # Report progress
        E = E_new
        if display > 1:
            print('epoch = %d : E = %12.10f'% (i+1, E * scale))

    if i == maxiter-1:
        print('Warning: maxiter exceeded. Sammon mapping may not have converged...')

    # Fiddle stress to match the original Sammon paper
    E = E * scale
    
    return [y,E]
    
def sammon_plot(y,plot_labels,path):
    x_axis = []
    y_axis = []

    for i in range(len(y)):
        x_axis.append(y[i][0])
        y_axis.append(y[i][1])

    sns.scatterplot(x_axis, y_axis, hue=plot_labels, legend='full') 
    #images.append(path)
    plt.savefig(path)   
    plt.close()
    #plt.show()

In [15]:
def PCA_plot(x,y,n_dim,path):
    
    x = StandardScaler().fit_transform(x)
    pca = PCA(n_components=n_dim)
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents,columns = ['principal component 1', 'principal component 2'])
    finalDf = pd.concat([principalDf, pd.Series(y)], axis = 1)
    finalDf.columns = ['principal component 1', 'principal component 2','target']
    
    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    targets = [0,1]
    colors = ['r', 'g']
    for target, color in zip(targets,colors):
        indicesToKeep = finalDf['target'] == target
        ax.scatter(finalDf.loc[indicesToKeep, 'principal component 1']
                   , finalDf.loc[indicesToKeep, 'principal component 2']
                   , c = color)
    ax.legend(targets)
    #images.append(path)
    fig.savefig(path)   
    plt.close(fig)
    #plt.show()

In [18]:
def tsne_plot(x,y,path):
    tsne = TSNE()
    X_embedded = tsne.fit_transform(x)
    sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y, legend='full')
    #images.append(path)
    plt.savefig(path)   
    plt.close()

## Unsupervised

In [6]:
bat_len = len(get_gene_sequences("../datasets/bat_flu.fa"))
cat_len = len(get_gene_sequences("../datasets/cat_flu.fa"))
zeros = [0]*bat_len
labels_all = np.append(zeros, [1]*cat_len, axis=None)

In [5]:
df = get_kmer_table("../datasets/combined_Bat_Cat_flu.fa",2,5)
df

,aa,aaa,aaaa,aaaaa,aaaac,aaaag,aaaat,aaac,aaaca,aaacc,...,tttg,tttga,tttgc,tttgg,tttgt,tttt,tttta,ttttc,ttttg,ttttt
0,0.163382,0.059133,0.023215,0.006570,0.002190,0.005694,0.008760,0.007884,0.003942,0.002190,...,0.003504,0.002190,0.000438,0.000876,0.000000,0.004818,0.002190,0.001752,0.000438,0.000438
1,0.150594,0.057684,0.019815,0.005284,0.002642,0.004844,0.007045,0.009247,0.006605,0.001761,...,0.005724,0.002642,0.000440,0.000881,0.001761,0.007486,0.002202,0.001321,0.002202,0.001761
2,0.164332,0.063025,0.020542,0.007937,0.004669,0.004202,0.003735,0.010271,0.004202,0.002801,...,0.005135,0.003268,0.000467,0.000934,0.000467,0.006536,0.001867,0.002801,0.000934,0.000934
3,0.148280,0.058126,0.022539,0.006524,0.003559,0.005338,0.007117,0.010083,0.005931,0.001779,...,0.006524,0.001779,0.001186,0.002372,0.001186,0.008897,0.001779,0.002372,0.002966,0.001779
4,0.133869,0.048193,0.014726,0.002677,0.003347,0.004016,0.004685,0.006693,0.002677,0.002677,...,0.007363,0.005355,0.000000,0.002008,0.000000,0.004016,0.000669,0.001339,0.002008,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,0.119607,0.036309,0.009825,0.002563,0.000854,0.002136,0.004272,0.008116,0.003845,0.002136,...,0.003845,0.001282,0.000427,0.000854,0.001282,0.002136,0.000000,0.001282,0.000000,0.000854
174,0.119944,0.040446,0.012552,0.002325,0.001860,0.003719,0.004649,0.006509,0.003254,0.002325,...,0.005579,0.002325,0.000930,0.000930,0.001395,0.005579,0.000930,0.001395,0.001395,0.001860
175,0.113463,0.038801,0.013521,0.001176,0.002939,0.004115,0.005291,0.008230,0.002352,0.002939,...,0.005879,0.001176,0.001764,0.001764,0.001176,0.005291,0.000588,0.001176,0.002352,0.001176
176,0.091489,0.021277,0.004965,0.001418,0.000000,0.001418,0.002128,0.002837,0.001418,0.001418,...,0.008511,0.001418,0.003546,0.002128,0.001418,0.003546,0.001418,0.000709,0.000709,0.000709


In [53]:
path = "../datasets/combined_Bat_Cat_flu.fa"
k_min = 2 
k_max = 5 
num_class = 2 
cov_type = 'full' 
seed = 0
predictions1 = get_predictions(path,k_min,k_max,num_class,cov_type,seed)

In [45]:
number = list(range(1, len(predictions1)+1))
df = pd.DataFrame(list(zip(number, predictions1)), 
               columns =['Number', 'Labels']) 
df.to_csv('predictions_unsup.csv',index = False)

In [54]:
cal_accuracy(labels_all,predictions1)

0.7921348314606742

In [13]:
df2 = df.drop_duplicates(keep="last")
X = df2.as_matrix(columns = df2.columns)
[y, E] = sammon(X,2)
sammon_plot(y,predictions1[df2.index.values],'figures/sammon_0.png')

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


epoch = 1 : E = 0.1707774274
epoch = 2 : E = 0.1682844665
epoch = 3 : E = 0.1680788403
epoch = 4 : E = 0.1473698705
epoch = 5 : E = 0.1382653352
epoch = 6 : E = 0.1380685295
epoch = 7 : E = 0.1339020615
epoch = 8 : E = 0.1299775413
epoch = 9 : E = 0.1228088618
epoch = 10 : E = 0.1227708633
epoch = 11 : E = 0.1183179845
epoch = 12 : E = 0.1047686226
epoch = 13 : E = 0.1044749198
epoch = 14 : E = 0.0714260052
epoch = 15 : E = 0.0686949361
epoch = 16 : E = 0.0686131084
epoch = 17 : E = 0.0593680302
epoch = 18 : E = 0.0430239338
epoch = 19 : E = 0.0428067416
epoch = 20 : E = 0.0422580956
epoch = 21 : E = 0.0402776300
epoch = 22 : E = 0.0388993272
epoch = 23 : E = 0.0385171135
epoch = 24 : E = 0.0373114071
epoch = 25 : E = 0.0372411020
epoch = 26 : E = 0.0369527426
epoch = 27 : E = 0.0367705021
epoch = 28 : E = 0.0366992383
epoch = 29 : E = 0.0365476292
epoch = 30 : E = 0.0365177142
epoch = 31 : E = 0.0363428715
epoch = 32 : E = 0.0362508610
epoch = 33 : E = 0.0362083009
epoch = 34 : E = 0.

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [14]:
sammon_plot(y,labels_all[df2.index.values],'figures/sammon_labels.png')

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [16]:
PCA_plot(df,predictions1,2,'figures/pca_0.png')

In [17]:
PCA_plot(df,labels_all,2,'figures/pca_labels.png')

In [19]:
tsne_plot(df,predictions1,'figures/tsne_0.png')

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [21]:
tsne_plot(df,labels_all,'figures/tsne_labels.png')

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


# Semi-supervised 50%

In [56]:
labels_50 = pd.read_csv('../datasets/labels_fifty_percent.csv')
labels_50 = pd.Series(labels_50['Labels'])
path = "../datasets/combined_Bat_Cat_flu.fa"
predictions_50 = model_selection(path,labels_50,2)

Best model has the following parameters:
minimum length of kmer:  2
maximum length of kmer:  4
covariance type:  diag
It has an accuracy regard to known labels of  0.9101123595505618


In [57]:
df = get_kmer_table("../datasets/combined_Bat_Cat_flu.fa",2,4)

In [58]:
df2 = df.drop_duplicates(keep="last")
X = df2.as_matrix(columns = df2.columns)
[y, E] = sammon(X,2)

epoch = 1 : E = 0.1501982542
epoch = 2 : E = 0.1390477228
epoch = 3 : E = 0.1309532005
epoch = 4 : E = 0.1105133031
epoch = 5 : E = 0.1086170733
epoch = 6 : E = 0.1059069652
epoch = 7 : E = 0.1027502562
epoch = 8 : E = 0.0997626733
epoch = 9 : E = 0.0981661969
epoch = 10 : E = 0.0810052761
epoch = 11 : E = 0.0750417553
epoch = 12 : E = 0.0676841812
epoch = 13 : E = 0.0443979286
epoch = 14 : E = 0.0413650252
epoch = 15 : E = 0.0383409852
epoch = 16 : E = 0.0358561901
epoch = 17 : E = 0.0352874998
epoch = 18 : E = 0.0352676026
epoch = 19 : E = 0.0347272441
epoch = 20 : E = 0.0343248301
epoch = 21 : E = 0.0334448992
epoch = 22 : E = 0.0327699411
epoch = 23 : E = 0.0323644999
epoch = 24 : E = 0.0322170766
epoch = 25 : E = 0.0321071341
epoch = 26 : E = 0.0320358824
epoch = 27 : E = 0.0319015036
epoch = 28 : E = 0.0318296209
epoch = 29 : E = 0.0317774122
epoch = 30 : E = 0.0317210447
epoch = 31 : E = 0.0315278575
epoch = 32 : E = 0.0315148521
epoch = 33 : E = 0.0314928474
epoch = 34 : E = 0.

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [59]:
number = list(range(1, len(predictions_50)+1))
df = pd.DataFrame(list(zip(number, predictions_50)), 
               columns =['Number', 'Labels']) 
df.to_csv('predictions_50.csv',index = False)

In [60]:
cal_accuracy(labels_50,predictions_50)

0.9101123595505618

In [61]:
sammon_plot(y,predictions_50[df2.index.values],'figures/sammon_50.png')
PCA_plot(df,predictions_50,2,'figures/pca_50.png')
tsne_plot(df,predictions_50,'figures/tsne_50.png')

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


# Semi-supervised 10%

In [63]:
labels_10 = pd.read_csv('../datasets/labels_ten_percent.csv')
labels_10 = pd.Series(labels_10['Labels'])
path = "../datasets/combined_Bat_Cat_flu.fa"

predictions_10 = model_selection(path,labels_10,2)

Best model has the following parameters:
minimum length of kmer:  2
maximum length of kmer:  4
covariance type:  diag
It has an accuracy regard to known labels of  0.9411764705882353


In [64]:
number = list(range(1, len(predictions_10)+1))
df = pd.DataFrame(list(zip(number, predictions_10)), 
               columns =['Number', 'Labels']) 
df.to_csv('predictions_10.csv',index = False)

In [65]:
df = get_kmer_table("../datasets/combined_Bat_Cat_flu.fa",2,4)
df2 = df.drop_duplicates(keep="last")
X = df2.as_matrix(columns = df2.columns)
[y, E] = sammon(X,2)

epoch = 1 : E = 0.1501982542
epoch = 2 : E = 0.1390477228
epoch = 3 : E = 0.1309532005
epoch = 4 : E = 0.1105133031
epoch = 5 : E = 0.1086170733
epoch = 6 : E = 0.1059069652
epoch = 7 : E = 0.1027502562
epoch = 8 : E = 0.0997626733
epoch = 9 : E = 0.0981661969
epoch = 10 : E = 0.0810052761
epoch = 11 : E = 0.0750417553
epoch = 12 : E = 0.0676841812
epoch = 13 : E = 0.0443979286
epoch = 14 : E = 0.0413650252
epoch = 15 : E = 0.0383409852
epoch = 16 : E = 0.0358561901
epoch = 17 : E = 0.0352874998
epoch = 18 : E = 0.0352676026
epoch = 19 : E = 0.0347272441
epoch = 20 : E = 0.0343248301
epoch = 21 : E = 0.0334448992
epoch = 22 : E = 0.0327699411
epoch = 23 : E = 0.0323644999
epoch = 24 : E = 0.0322170766
epoch = 25 : E = 0.0321071341
epoch = 26 : E = 0.0320358824
epoch = 27 : E = 0.0319015036
epoch = 28 : E = 0.0318296209
epoch = 29 : E = 0.0317774122
epoch = 30 : E = 0.0317210447
epoch = 31 : E = 0.0315278575
epoch = 32 : E = 0.0315148521
epoch = 33 : E = 0.0314928474
epoch = 34 : E = 0.

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [66]:
cal_accuracy(labels_10,predictions_10)

0.9411764705882353

In [67]:
sammon_plot(y,predictions_10[df2.index.values],'figures/sammon_10.png')
PCA_plot(df,predictions_10,2,'figures/pca_10.png')
tsne_plot(df,predictions_10,'figures/tsne_10.png')

/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
/Users/zhiwen/Applications/miniconda3/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning
